# Rotor forces and torques (symbolic)

## 1. Set up the notebook

Do imports.

In [1]:
import numpy as np
import sympy as sym
from IPython.display import display, Math

## 2. Show how to take cross products with sympy

Suppose we want to take the cross product
$ w \times v $
where $w$ and $v$ are described in coordinates as follows:

$$w^B = \begin{bmatrix} a \\ b \\ c \end{bmatrix} \qquad\qquad v^B = \begin{bmatrix} d \\ e \\ f \end{bmatrix}.$$

First, create symbolic variables.

In [2]:
a, b, c, d, e, f = sym.symbols('a, b, c, d, e, f')

Second, create $w$ and $v$ as sympy matrices:

In [3]:
w_inB = sym.Matrix([a, b, c])
v_inB = sym.Matrix([d, e, f])

Now, the easiest way to take the cross product is with [sympy.cross](https://docs.sympy.org/latest/modules/matrices/matrices.html#sympy.matrices.matrices.MatrixBase.cross).

In [4]:
w_inB.cross(v_inB)

Matrix([
[ b*f - c*e],
[-a*f + c*d],
[ a*e - b*d]])

An alternative is to use the wedge operator. If

$$ w^B = \begin{bmatrix} a \\ b \\ c \end{bmatrix} $$

then

$$ \widehat{w^B} = \begin{bmatrix} 0 & -c & b \\ c & 0 & -a \\ -b & a & 0 \end{bmatrix} $$

and the cross product $w \times v$ in the coordinates of frame $B$ can be found by matrix multiplication:

$$ \left( w \times v \right)^B = \widehat{w^B} v^b. $$

The "wedge operator" is the mapping from the column vector $w^B$ to the skew-symmetric matrix $\widehat{w^B}$. The function [sympy.hat](https://docs.sympy.org/latest/modules/matrices/matrices.html#sympy.matrices.matrixbase.MatrixBase.hat) implements the wedge operator.

In [5]:
w_inB.hat()

Matrix([
[ 0, -c,  b],
[ c,  0, -a],
[-b,  a,  0]])

Here is how we would use the wedge operator to take the same cross product as before.

In [6]:
w_inB.hat() @ v_inB

Matrix([
[ b*f - c*e],
[-a*f + c*d],
[ a*e - b*d]])

Make sure the result is the same.

In [7]:
assert(sym.simplify(w_inB.cross(v_inB) - w_inB.hat() @ v_inB) == sym.zeros(3, 1))

## 3. Derive relationship between motor power commands and forces and torques

Define parameters as symbolic variables.

In [8]:
k_F, k_M, l = sym.symbols('k_F, k_M, l')

Define motor power commands as symbolic variables.

In [9]:
m_1, m_2, m_3, m_4 = sym.symbols('m_1, m_2, m_3, m_4')

Find net force and torque from the first motor.

In [10]:
# Net force
f_inB_of1 = sym.Matrix([0, 0, k_F * m_1])

# Point at which force is applied
p_inB_of1 = sym.Matrix([l, -l, 0])

# Net torque
tau_inB_of1 = sym.Matrix([0, 0, -k_M * m_1]) + p_inB_of1.cross(f_inB_of1)

Show the result.

In [11]:
print('net force from rotor 1 (in the coordinates of frame B):')
display(f_inB_of1)
print('net torque from rotor 1  (in the coordinates of frame B):')
display(tau_inB_of1)

net force from rotor 1 (in the coordinates of frame B):


Matrix([
[      0],
[      0],
[k_F*m_1]])

net torque from rotor 1  (in the coordinates of frame B):


Matrix([
[-k_F*l*m_1],
[-k_F*l*m_1],
[  -k_M*m_1]])

Find net force and torque from all other motors.

In [12]:
# Motor 2
f_inB_of2 = sym.Matrix([0, 0, k_F * m_2])
p_inB_of2 = sym.Matrix([-l, -l, 0])
tau_inB_of2 = sym.Matrix([0, 0, k_M * m_2]) + p_inB_of2.cross(f_inB_of2)

# Motor 3
f_inB_of3 = sym.Matrix([0, 0, k_F * m_3])
p_inB_of3 = sym.Matrix([-l, l, 0])
tau_inB_of3 = sym.Matrix([0, 0, -k_M * m_3]) + p_inB_of3.cross(f_inB_of3)

# Motor 4
f_inB_of4 = sym.Matrix([0, 0, k_F * m_4])
p_inB_of4 = sym.Matrix([l, l, 0])
tau_inB_of4 = sym.Matrix([0, 0, k_M * m_4]) + p_inB_of4.cross(f_inB_of4)

Find total force and torque from all motors.

In [13]:
f_inB_ofRotors = f_inB_of1 + f_inB_of2 + f_inB_of3 + f_inB_of4
tau_inB_ofRotors = tau_inB_of1 + tau_inB_of2 + tau_inB_of3 + tau_inB_of4

Show the result.

In [14]:
print('net force from all rotors (in the coordinates of frame B):')
display(f_inB_ofRotors)
print('net torque from all rotors  (in the coordinates of frame B):')
display(tau_inB_ofRotors)

net force from all rotors (in the coordinates of frame B):


Matrix([
[                                    0],
[                                    0],
[k_F*m_1 + k_F*m_2 + k_F*m_3 + k_F*m_4]])

net torque from all rotors  (in the coordinates of frame B):


Matrix([
[-k_F*l*m_1 - k_F*l*m_2 + k_F*l*m_3 + k_F*l*m_4],
[-k_F*l*m_1 + k_F*l*m_2 + k_F*l*m_3 - k_F*l*m_4],
[        -k_M*m_1 + k_M*m_2 - k_M*m_3 + k_M*m_4]])

Define the matrix $P$ that maps motor power commands to net forces and torques:

$$
\begin{bmatrix} \tau_x \\ \tau_y \\ \tau_z \\ f_z \end{bmatrix}
=
P
\begin{bmatrix} m_1 \\ m_2 \\ m_3 \\ m_4 \end{bmatrix}.
$$

In [15]:
P = sym.Matrix([
    [tau_inB_ofRotors[0].coeff(a) for a in [m_1, m_2, m_3, m_4]],
    [tau_inB_ofRotors[1].coeff(a) for a in [m_1, m_2, m_3, m_4]],
    [tau_inB_ofRotors[2].coeff(a) for a in [m_1, m_2, m_3, m_4]],
    [f_inB_ofRotors[2].coeff(a) for a in [m_1, m_2, m_3, m_4]],
])

Show the result.

In [25]:
display(Math(r'P=' + sym.latex(P)))

<IPython.core.display.Math object>

Find the matrix $P^{-1}$ that maps net forces and torques to motor power commands:

$$
\begin{bmatrix} m_1 \\ m_2 \\ m_3 \\ m_4 \end{bmatrix}
=
P^{-1}
\begin{bmatrix} \tau_x \\ \tau_y \\ \tau_z \\ f_z \end{bmatrix}.
$$

In [17]:
Pinv = P.inv()

Show the result.

In [26]:
display(Math(r'P^{-1}=' + sym.latex(Pinv)))

<IPython.core.display.Math object>

Show the result again in a way that is easier to read.

In [24]:
display(Math(r'P^{-1}=' + sym.latex(Pinv, mat_str='bmatrix', mat_delim='').replace('frac', 'dfrac').replace('\\\\', '\\\\[1.5em]')))

<IPython.core.display.Math object>